In [1]:
import pandas as pd 
import numpy as np 
import datetime as dt 
import requests
import warnings
import json
import re
from bs4 import BeautifulSoup

<h1 style="color:turquoise">Get Data from API</h1>

In [8]:
"""
Potentially useful data
SME Export Value: https://data.go.th/dataset/export2556-2566
SME Import Value: https://data.go.th/dataset/2556-2566
"""

'\nPotentially useful data\nSME Export Value: https://data.go.th/dataset/export2556-2566\nSME Import Value: https://data.go.th/dataset/2556-2566\n'

In [8]:
def crawl_url_raw(url):
    # Send a GET request to the URL and store the response
    response = requests.get(url)

    # Use BeautifulSoup to parse the HTML content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all div elements with class "row hoverDownload" or "row hoverDownload active"
    divs = soup.find_all('div', {'class': ['row hoverDownload', 'row hoverDownload active']})
    return divs

def crawl_url(url):
    # Send a GET request to the URL and store the response
    response = requests.get(url)

    # Use BeautifulSoup to parse the HTML content of the response
    soup = BeautifulSoup(response.content, 'html.parser')

    # Find all div elements with class "row hoverDownload" or "row hoverDownload active"
    divs = soup.find_all('div', {'class': ['row hoverDownload', 'row hoverDownload active']})

    # Extract the link and title from each div
    results = list()
    # global div
    for div in divs:
        # check if the file is in .xlsx format. if so, get the url
        div = [r for r in div]
        if div[1].find('a').find('img')['alt'] == 'xlsx':
            # name = div[3].find('a')['title'] or div[3].find('a').title
            url = div[1].find('a')['href']
            # print(f'Name = {name} and url = {url}')
            # results[name] = url
            results.append(url)

    # Return the extracted data
    return results


In [9]:
res_list = crawl_url('https://data.go.th/dataset/export2556-2566')

In [4]:
# res_list[:5]

In [5]:
# for each data year, get only first element found (since it's the latest update)
r = r'opendata-export[A-Za-z0-9-.]*'
year_dict = {url: re.findall(r, url)[0][15:19] for url in res_list}

latest_update = dict()
for url, year in year_dict.items():
    if year not in latest_update:
        latest_update[year] = url

latest_update

In [6]:
export_dir = './data/thai_gov_open_data/sme_export'
for year, url in latest_update.items():
    if year in []:
        continue
    print(f'Year = {year} and URL = {url}')
    try:
        # * get file format
        file_format = url.split('.')[-1]
        response = requests.get(url)
        if response.status_code == 200:
            if file_format == 'xlsx':
                df = pd.read_excel(response.content)
                df.to_csv(f'{export_dir}/{str(year)}.csv', index=False)
            elif file_format == 'csv':
                df = pd.read_csv(response.content)
                df.to_csv(f'{export_dir}/{str(year)}.csv', index=False)
            else:
                warnings.warn(f'File format {file_format} not supported', category=UserWarning)
    except:
        print(f'Failed to download from {year}')

<h1 style="color:turquoise">Convert Data from CSV to Parquet</h1>

In [37]:
base_path = './data/thai_gov_open_data/sme_export'
for year in range(2564, 2567):
    path = f'{base_path}/csv/{str(year)}.csv'
    new_path = f'{base_path}/parquet/{str(year)}.parquet'
    df = pd.read_csv(path, skiprows=0)
    df.columns = [c.strip() for c in df.columns]
    df['มูลค่า (บาท)'] = df['มูลค่า (บาท)'].astype(float)
    df['มูลค่า (ดอลล่าร์)'] = df['มูลค่า (ดอลล่าร์)'].astype(float)
    df.to_parquet(new_path, index=False)
    print(f'{str(year)} is completed')

2564 is completed
2565 is completed
2566 is completed


In [29]:
# base_path = './data/thai_gov_open_data/sme_export'
# path = f'{base_path}/2556.csv'
# df = pd.read_csv(path, skiprows=1)
# df.head(2)

<h1 style="color:turquoise">Read Data</h1>

In [3]:
# read data
sme_export_raw = pd.read_parquet('./data/thai_gov_open_data/sme_export/parquet/')
# sme_export_raw = pq.read_table('./data/thai_gov_open_data/sme_export/parquet/')
sme_export_raw.columns = ['country', 'hs2dg', 'hs2dg_desc', 'hs4dg', 'hs4dg_desc', 'business_size', 'value_thb', 'value_usd', 'month', 'year']

sme_export_raw['month'] = sme_export_raw.apply(lambda row: dt.date(row['year'], row['month'], 1), axis = 1)

sme_export_raw.set_index('month', inplace=True)

In [4]:
sme_export_raw['year'].unique()

array([2556, 2557, 2558, 2559, 2560, 2561, 2562, 2563, 2564, 2565, 2566])

In [5]:
# # * total export value by countries and months
country = sme_export_raw[['country', 'value_thb']] \
            .groupby([sme_export_raw.index, 'country']).sum()

# ? divide value_thb by a billion to make it easier to visualize
country['value_thb'] = country['value_thb'].div(1e9)

country['monthly_rank'] = country.groupby([country.index.get_level_values(0)])['value_thb'].rank(method='max', ascending=False).astype(int)

# * top 5 each month
country_top5 = country[country['monthly_rank'] <= 5]
country_top5 = country_top5.sort_values(by=['month', 'monthly_rank'])

In [6]:
country_top5.head(20)

value_thb  monthly_rank
month      country                               
2556-01-01 CHINA           2.195181             1
           JAPAN           1.813440             2
           UNITED STATES   1.812230             3
           HONG KONG       1.041262             4
           MALAYSIA        1.030493             5
2556-02-01 CHINA           2.234773             1
           JAPAN           1.805094             2
           UNITED STATES   1.794485             3
           INDONESIA       0.997919             4
           HONG KONG       0.973395             5
2556-03-01 CHINA           2.397601             1
           JAPAN           1.988730             2
           UNITED STATES   1.900329             3
           HONG KONG       1.358262             4
           MALAYSIA        1.114512             5
2556-04-01 CHINA           2.041396             1
           UNITED STATES   1.775266             2
           JAPAN           1.736530             3
           MALAYSIA        0.986922             4
           INDONESIA       0.963330             5

In [7]:
# sme_export_raw.index

In [3]:
# tmp56 = pd.read_parquet('./data/thai_gov_open_data/sme_export/2556.parquet')
# tmp56.head()